In [5]:
import matplotlib, time, copy
import matplotlib.pyplot as plt
import autograd.numpy as np
import autograd.scipy.stats as sps_autograd
from autograd import grad, hessian
from statsmodels.tsa.arima_process import ArmaProcess

# Simulate ARMA data 

In [6]:
# Simulate ARMA(3, 2) model

# Define AR and MA coefficients
ar = np.array([1, -0.5, 0.3, -0.2])  
ma = np.array([1, 0.4, -0.2])        

# Create ARMA process object
arma_process = ArmaProcess(ar, ma)

# Simulate 1000 samples
N = 1000
y = arma_process.generate_sample(nsample=N)

# Karman Filter 

In [7]:
def initialize_FGHQ(a, b):
    """
    Construct the state-space matrices F, G, H for an ARMA(p, q) model.

    Parameters:
    - p: int, order of the AR component
    - q: int, order of the MA component
    - a: list or np.array of AR coefficients [a1, a2, ..., ap]
    - b: list or np.array of MA coefficients [b1, b2, ..., bq]

    Returns:
    - F: state transition matrix of shape (k, k)
    - G: noise coefficient matrix of shape (k, 1)
    - H: observation matrix of shape (1, k)
    - Q: covariance identity matrix of shape (k, k)
    """
    p = len(a)
    q = len(b)
    k = max(p, q + 1)  # dimension of the state vector
    F = np.zeros((k, k))
    G = np.zeros((k, 1))
    H = np.zeros((1, k))

    # Fill the first column of F with AR coefficients
    for i in range(p):
        F[i, 0] = a[i]

    # Fill the lower subdiagonal of F with 1s (shifting the state)
    for i in range(k - 1):
        F[i, i + 1] = 1

    # Fill G with negative MA coefficients, first element = 1
    for i in range(min(q, k - 1)):
        G[i, 0] = -b[i]
    G[0, 0] = 1  # first element always set to 1

    # Observation matrix H: only first element is 1
    H[0, 0] = 1

    # Initialize covariance matrix Q as identity matrix
    Q = np.eye(k)

    return F, G, H, Q

In [8]:
# Initialize values
a = [0.2, 0.1, 0.3]
b = [0.1, 0.1]
k = max(len(a), len(b) + 1)

x = np.zeros((k, 1))
V = np.eye(k)
e = np.zeros((N, 1))
r = np.zeros((N, 1))

F, G, H, Q = initialize_FGHQ(a, b)

# Check values
print(F)
print(G)
print(H)
print(Q)

[[0.2 1.  0. ]
 [0.1 0.  1. ]
 [0.3 0.  0. ]]
[[ 1. ]
 [-0.1]
 [ 0. ]]
[[1. 0. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [9]:
for t in range(N):
    # Predict one-step-ahead state predictive density of x_{t}
    x_predict = F @ x       
    V_predict = F @ V @ F.T + G @ G.T  

    # Compute forecast error and one-step-ahead predictive variance
    e[t] = y[t] - (H @ x_predict).item()
    r[t] = (H @ V_predict @ H.T).item()

    # Kalman gain
    K = V_predict @ H.T / r[t]

    # Update current state and covariance
    x = x_predict + K * e[t]
    V = (np.eye(k) - K @ H) @ V_predict

# Calculate Log-likelihood

In [10]:
def log_likelihood(sigma_hat, r):
    N = len(r)
    return -0.5 * (N * np.log(2 * np.pi) 
                   + N * np.log(sigma_hat) 
                   + np.sum(np.log(r)) + N)

In [11]:
# Calculate sigma_hat
sigma_hat = np.sum(e**2 / r) / N

# Calculate log-likelihood
log_lik = log_likelihood(sigma_hat, r)

print("Log-likelihood:", log_lik)

Log-likelihood: -1760.3568983275975
